# Libraries

In [1]:
import tkinter as tk
from tkinter import filedialog,ttk
import cv2
from PIL import Image,ImageTk
import numpy as np
import matplotlib.pyplot as plt

# Desktop Application

In [2]:
def toGreyScale(image):
        if len(image.shape) != 3:
            print("input image must be a 3D color image.")
            return
        if image.shape[2] != 3:
            print("input image must have 3 channels (RGB).")
            return
        grayscale_image = 0.299 * image[:, :, 0] + 0.587 * image[:, :, 1] + 0.114 * image[:, :, 2]
        return grayscale_image.astype(np.uint8)
    
class gui:
    def __init__(self,root):
        self.root=root
        self.root.title("GUI")
        self.root.configure(bg="#7B5742")
        self.path=None
        self.index=0
        self.obj=None 
        self.readimage=None
        
        #Designing and choosing colours and positions of buttons
        #For input image
        self.input=tk.Frame(self.root,bg="#D2B48C")
        self.input.grid(row=0,column=0,padx=10,pady=10,sticky="nsew")
        self.label=tk.Label(self.input,text="Input Image/Video",bg="#D2B48C")
        self.label.grid(row=0,column=0,pady=5)
        self.upload=tk.Button(self.input,text="Upload Image/Video",command=self.upload,bg="#7B5742",fg="white")
        self.upload.grid(row=1,column=0,pady=5)

        #For output image
        self.output=tk.Frame(self.root,bg="#D2B48C")
        self.output.grid(row=0,column=1,padx=10,pady=10,sticky="nsew")
        self.olabel=tk.Label(self.output,text="Output Image/Video",bg="#D2B48C")
        self.olabel.grid(row=0,column=0,pady=5)
        self.ocanvas=tk.Canvas(self.output,width=400,height=300,bg="#E0C9A0")
        self.ocanvas.grid(row=1,column=0,pady=5)

        #For dropdown and go button
        self.dropdown=tk.Frame(self.root,bg="#D2B48C")
        self.dropdown.grid(row=1,column=0,columnspan=2,padx=10,pady=10,sticky="nsew")
        self.plabel=tk.Label(self.dropdown,text="Processing Technique : ",bg="#D2B48C")
        self.plabel.grid(row=0,column=0,padx=5,pady=5)
        self.options=["Thresholding","Histogram Equalization","Averaging Filter","Laplacian","adaptiveThresholdingCustom","kMeansClustringCustom","LaplacianOfGausianCustom"]
        self.p=ttk.Combobox(self.dropdown,values=self.options)
        self.p.grid(row=0,column=1,padx=5,pady=5)
        self.gobutton=tk.Button(self.dropdown,text="Go",command=self.process,bg="#7B5742",fg="white")
        self.gobutton.grid(row=0,column=2,padx=5,pady=5)

        #For next and previous frame button
        self.frame=tk.Frame(self.root,bg="#D2B48C")
        self.frame.grid(row=2,column=0,columnspan=2,padx=10,pady=10,sticky="nsew")
        self.prev=tk.Button(self.frame,text="Previous Frame",command=self.prevframe,bg="#7B5742",fg="white")
        self.prev.grid(row=0,column=0,padx=5)
        self.next= tk.Button(self.frame,text="Next Frame",command=self.nextframe,bg="#7B5742",fg="white")
        self.next.grid(row=0,column=1,padx=5)
        
    def upload(self):
        self.path=filedialog.askopenfilename()
        if self.path:
            self.obj=cv2.VideoCapture(self.path)
            if self.obj.isOpened():
                self.index=0
                self.displayinput()
                self.process()

    def displayinput(self):
        flag,frame=self.obj.read()
        self.readimage=frame
        if flag==True:
            frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            frame=Image.fromarray(frame)
            frame.thumbnail((300,300))
            frame=ImageTk.PhotoImage(frame)
            self.label.config(image=frame)
            self.label.image=frame

    def process(self):
        selected=self.p.get()
        if not selected:
            return
        flag,frame=self.obj.read()
        if not flag:
            if selected=="Thresholding":
                p=self.thresholding(self.readimage)
            elif selected=="Histogram Equalization":
                p=self.histogram(self.readimage)
            elif selected=="Averaging Filter":
                p=self.averaging(self.readimage)
            elif selected=="Laplacian":
                p=self.laplacian(self.readimage)
            elif selected=="adaptiveThresholdingCustom":
                p=self.adaptiveThreshold(self.readimage)
            elif selected=="kMeansClustringCustom":
                p=self.kMeansClustring(self.readimage, 3)
            elif selected=="LaplacianOfGausianCustom":
                p=self.laplacianCustom(self.readimage)
            self.displayoutput(p)
        else:
            if selected=="Thresholding":
                p=self.thresholding(frame)
            elif selected=="Histogram Equalization":
                p=self.histogram(frame)
            elif selected=="Averaging Filter":
                p=self.averaging(frame)
            elif selected=="Laplacian":
                p=self.laplacian(frame)
            elif selected=="adaptiveThresholdingCustom":
                p=self.adaptiveThreshold(frame)
            elif selected=="kMeansClustringCustom":
                p=self.kMeansClustring(frame, 3)
            elif selected=="LaplacianOfGausianCustom":
                p=self.laplacianCustom(frame)
            self.displayoutput(p)

    def kMeansClustring(self, image, k, maxIters=100):
        data = np.float32(image.reshape((-1, 3)))

        # Function to perform K-means clustering
        def clustering(data, k, maxIters):
            #randomly select k indices as initial centroids
            indices = np.random.choice(data.shape[0], k, replace=False)
            centroids = data[indices]

            for _ in range(maxIters):
                distances = np.sqrt(((data[:, None, :] - centroids[None, :, :])**2).sum(axis=2))
                #assign labels based on closest centroid
                labels = np.argmin(distances, axis=1)
                newCentroids = np.array([data[labels == i].mean(axis=0) for i in range(k)])
                #terminating condition (Mean(x-1)==Mean(x))
                if np.allclose(centroids, newCentroids):
                    break
                centroids = newCentroids
            return centroids, labels

        centroids, labels = clustering(data, k, maxIters)
        segmentedData = centroids[labels].reshape(image.shape)
        segmentedData = segmentedData.astype(np.uint8)
        return segmentedData
    
    def adaptiveThreshold(self, image, blockSize = 11, constant = 2):
        image = toGreyScale(image)
        height, width = image.shape
        thresholdedImage = np.zeros_like(image, dtype=np.uint8)#empty output image 
        #iterate through each pixel (excluding borders for complete neighborhood)
        for row in range(blockSize // 2, height - blockSize // 2):
            for col in range(blockSize // 2, width - blockSize // 2):
                #to get neighborhood around pixel
                neighborhood = image[row - blockSize // 2 : row + blockSize // 2 + 1,
                                   col - blockSize // 2 : col + blockSize // 2 + 1]
                localThreshold = np.mean(neighborhood) - constant  
                thresholdedImage[row, col] = 255 if image[row, col] > localThreshold else 0 # Apply thresholding
        return thresholdedImage

    def laplacianCustom(self, image): 
        image = toGreyScale(image)
        kernel = np.array([[0, 1, 0],
                           [1,-4, 1],
                           [0, 1, 0]])
        #pading image with zeros to handle boundary effects
        paddedImage = np.pad(image, pad_width=1, mode='constant')
        #performing convolution using element-wise multiplication and summation
        laplacianImage = np.zeros_like(image)
        for row in range(1, image.shape[0] + 1):
            for col in range(1, image.shape[1] + 1):
                window = paddedImage[row-1:row+2, col-1:col+2]
                laplacianImage[row-1, col-1] = np.sum(window * kernel)
        return laplacianImage

    def thresholding(self,frame):
        frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        _,frame=cv2.threshold(frame,127,255,cv2.THRESH_BINARY)
        return frame

    def histogram(self,frame):
        frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        frame=cv2.equalizeHist(frame)
        return frame

    def averaging(self, frame):
        frame=cv2.blur(frame,(5,5))
        return frame
    
    def laplacian(self,frame):
        frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        frame=cv2.Laplacian(frame,cv2.CV_64F)
        frame=np.uint8(np.absolute(frame))
        return frame
    
    def displayoutput(self,frame):
        w=self.label.winfo_width()
        h=self.label.winfo_height()
        frame=cv2.resize(frame,(w,h))
        frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        frame=ImageTk.PhotoImage(image=Image.fromarray(frame))
        self.ocanvas.create_image(0,0,anchor="nw",image=frame)
        self.ocanvas.image=frame

    def nextframe(self):
        if self.obj:
            flag,frame=self.obj.read()
            if flag:
                self.displayinput()
                self.process()
                
    def prevframe(self):
        if self.obj:
            flag,frame=self.obj.read()
            if flag:
                self.displayinput()
                self.process()


if __name__=="__main__":
    root=tk.Tk()
    app=gui(root)
    root.mainloop()


C:\Users\hassa\AppData\Local\Temp\ipykernel_11720\2188538733.py:128: RuntimeWarning: Mean of empty slice.
  newCentroids = np.array([data[labels == i].mean(axis=0) for i in range(k)])
C:\Users\hassa\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


---

**Note:** We have tried our best to provide accurate results in this notebook. However, these results may not be entirely accurate, and contributions or corrections are encouraged. Thank you!
